<a href="https://colab.research.google.com/github/GVGunasekhar/Binance-trade-analysis/blob/main/TEXT_CLASSIFICATION_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m -m spacy download en_core_web_sm
!pip install fuzzywuzzy[speedup]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 106.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 62.9 MB/s eta 0:00:00


In [ ]:
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

# Load spaCy's small English model
nlp = spacy.load('en_core_web_sm')

# Sample data: Mapping towns/cities to districts
districts = {
    "Ananthapuram": ["Ananthapuramu", "Puttaparthi", "Kadiri", "Kalyandurg", "Rayadurg", "Dharmavaram", "Nallamada", "Bukkapatnam", "Peddapappur", "Singanamala", "Somanepalle", "Vemapalli", "Mudigubba"],
    "Chittoor": ["Chittoor", "Tirupati", "Puttur", "Kuppam", "Madanapalle", "Srikalahasti", "Palamaner", "Nagari", "Vayalpad", "Chandragiri", "Ramachandrapuram", "Mulbagal", "Bhakarapet", "Kolar", "Peddapappur"],
    "East Godavari": ["Kakinada", "Rajahmundry", "Amalapuram", "Peddapuram", "G.Ramachandrapuram", "Gokavaram", "Mandapeta", "Jaggampeta", "Samalkota", "Pithapuram", "Kirlampudi", "Ailavaram", "Tuni", "Kothapeta", "Karapa"],
    "Guntur": ["Guntur", "Tenali", "Narasaraopeta", "Bapatla", "Mangalagiri", "Ponnur", "Repalle", "Vinukonda", "Kollipara", "Duggirala", "Prathipadu", "Amaravati", "Sattenapalle", "Macherla", "Piduguralla"],
    "Kadapa": ["Proddatur", "Rayachoti", "Jammalamadugu", "Pulivendula", "Mydukur", "Kadapa", "Brahmamgarimatham", "Chakrayapeta", "Kamalapuram", "Peddamudiyam", "Rajupalem", "Lakkireddipalle", "Rudravaram"],
    "Krishna": ["Vijayawada", "Machilipatnam", "Gudivada", "Nuzvid", "Kanchikacherla", "Challapalli", "Penamaluru", "Tiruvuru", "Jaggaiahpet", "Marteru", "Madhuranagar", "Nandigama", "Qutubullapuram", "Kankipadu", "Kothapeta", "Jaggayyapeta", "Mylavaram", "Bapatla", "Pedana"],
    "Kurnool": ["Kurnool", "Adoni", "Nandikotkur", "Nallapadu", "Dhone", "Allagadda", "Banaganapalle", "Atmakur", "Gadivemula", "Yemmiganur", "Mantralayam", "Nandavaram", "Gonegandla", "Kallur", "Krishnagiri", "Kowthalam", "Mahanandi", "Nangali", "Palvapada"],
    "Nellore": ["Atmakur", "Chandra Sekhara Puram", "Duggirala", "Gudur", "Kavali", "Muthukur", "Narakodur", "Nellore", "Peddaganjam", "Sriharikota", "Sullurpet", "Thotapalligudem", "Udayagiri", "Venkatachalam", "Venkatagiri"],
    "Prakasam": ["Nagulapadu", "Darsi", "Sangam", "Peddagudi", "Giddalur", "Bapatla", "Kanigiri", "Ongole", "Markapur", "Ramapuram", "Chirala", "Podili", "Kandukur", "Peddaganjam", "Yerragondapalem", "Peddagollapalli", "Chandrasekharapuram", "Kothapatnam", "Muppalla"],
    "Srikakulam": ["Santabommali", "Hiramandalam", "Sompeta", "Vangara", "Etcherla", "Polaki", "Rajam", "Tekkali", "Srikakulam", "Palakonda", "Ichchapuram", "Narasannapeta", "Laveru", "Sarubujjili", "Kaviti", "Arasavalli", "Nellimarla"],
    "Visakhapatnam": ["Dattirajeru", "Yelamanchili", "Atchutapuram", "Thagarapuvalasa", "Payakaraopeta", "Tuni", "Vepada", "Chodavaram", "Visakhapatnam", "Peddagantyada", "Kothavalasa", "Anakapalle", "Madugula", "S. Kota", "Rambilli", "Narsipatnam", "Kasimkota", "Parawada", "Bheemunipatnam", "Adivarapupeta"],
    "Vizianagaram": ["Gummalakshmipuram", "Dattirajeru", "Peddagummanur", "S Kota", "Jiyyammavalasa", "Gajapathinagaram", "Parvathipuram", "Vizianagaram", "Nellimarla", "Borra", "Bobbili", "Balijipeta", "Kurupam", "Salur"],
    "West Godavari": ["Mogalthur", "Kalla", "Lakkavaram", "Palakollu", "Tadepalligudem", "Nallajerla", "Denduluru", "Kovvur", "Eluru", "Jangareddygudem", "Raghunadhapalem", "Bhimavaram", "Pedapadu", "Ganapavaram", "Bhimadole", "Narasapuram", "Akiveedu", "Gopalapuram", "Polavaram"]
}

# Function to detect named entities (NER) and extract city/town names
def detect_entity(input_text):
    input_doc = nlp(input_text)
    for ent in input_doc.ents:
        if ent.label_ in ["GPE", "LOC"]:  # GPE = Geopolitical entity, LOC = Location
            return ent.text
    return None

# Function to detect city/town using fuzzy matching and spaCy NER
def detect_city(input_text):
    city_town = detect_entity(input_text)

    if not city_town:  # If NER doesn't find anything, use fuzzy matching
        all_towns = [town for towns in districts.values() for town in towns]
        match = process.extractOne(input_text, all_towns, scorer=fuzz.token_set_ratio)

        if match:  # Check if match is not None
            city_town, score = match
            if score < 70:  # Adjusted threshold for matching accuracy
                city_town = None
        else:
            city_town = None

    return city_town

# Function to find the corresponding district
def find_district(city_town):
    for district, towns in districts.items():
        if city_town in towns:
            return district
    return None

# Initialize an empty DataFrame to hold all user inputs categorized by district
full_df = pd.DataFrame(columns=["User Input", "City/Town", "Predicted District",'Expected District'])

# Function to preprocess user input
def preprocess_input(input_text):
    return input_text.lower()

# Function to determine expected district from user input
def get_expected_district(input_text):
    input_text = preprocess_input(input_text)

    # Check if any town name is mentioned in the input to determine expected district
    for district, towns in districts.items():
        for town in towns:
            if town.lower() in input_text:
                return district
    return None  # If no match is found

# Function to classify user input based on city/town and store in DataFrame
def classify_and_store_input(input_text):
    expected_district = get_expected_district(input_text)  # Determine expected district
    processed_text = preprocess_input(input_text)
    city_town = detect_city(processed_text)
    predicted_district = find_district(city_town) if city_town else None

    new_entry = {
        "User Input": input_text,
        "City/Town": city_town,
        "Predicted District": predicted_district,
        "Expected District": expected_district
    }

    global full_df
    full_df = pd.concat([full_df, pd.DataFrame([new_entry])], ignore_index=True)

# Function to save the DataFrame to an Excel file
def save_to_excel():
    output_file = "UserInput_Output.xlsx"
    full_df.to_excel(output_file, index=False)
    print(f"Data saved to {output_file}")

# Function to calculate accuracy
def calculate_accuracy():
    correct_predictions = full_df[full_df['Predicted District'] == full_df['Expected District']]
    accuracy = (len(correct_predictions) / len(full_df)) * 100 if len(full_df) > 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")

# Main loop for user input
while True:
    user_input = input("Enter your details or describe something about a place (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    classify_and_store_input(user_input)

# After the loop, calculate accuracy and save the DataFrame
calculate_accuracy()
save_to_excel()


Enter your details or describe something about a place (or type 'exit' to quit): anjai from amalapuram
Enter your details or describe something about a place (or type 'exit' to quit): Anjani From Amalapuram
Enter your details or describe something about a place (or type 'exit' to quit): exit
Accuracy: 100.00%
Data saved to UserInput_Output.xlsx


#PREDICTED DISTRICT


In [ ]:
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd

# Load spaCy's small English model
nlp = spacy.load('en_core_web_sm')

# Load data from Excel
dataframe = pd.read_excel('/content/REGIONAL DATASET.xlsx')

# Print column names for debugging
print("Column names in the DataFrame:", dataframe.columns)

# Strip any whitespace from column names
dataframe.columns = dataframe.columns.str.strip()

data = {}

# Populate the data dictionary from the Excel sheet
for index, row in dataframe.iterrows():
    district = row['Districts']  # Updated to the correct column name
    cities = [city.strip().lower() for city in row['Cities/Towns/Mandals'].split(',')]  # Updated to the correct column name
    data[district] = cities

# Initialize an empty DataFrame to hold user input categorized by district
full_df = pd.DataFrame(columns=["User Input", "City/Town", "Predicted District"])

# Function to detect named entities (NER) and extract city/town names
def detect_entity(input_text):
    input_text = input_text.lower()  # Convert input to lowercase
    input_doc = nlp(input_text)
    for ent in input_doc.ents:
        if ent.label_ in ["GPE", "LOC"]:
            return ent.text
    return None

# Function to detect city/town using fuzzy matching and spaCy NER
def detect_city(input_text):
    input_text = input_text.lower()  # Convert input to lowercase
    city_town = detect_entity(input_text)

    if not city_town:  # If NER doesn't find anything, use fuzzy matching
        all_towns = [town.lower() for towns in data.values() for town in towns]  # Convert towns to lowercase
        match = process.extractOne(input_text, all_towns, scorer=fuzz.token_set_ratio)
        if match:  # Check if match is not None
            city_town, score = match
            if score < 80:  # Setting a threshold for matching accuracy
                city_town = None

    return city_town

# Function to find the corresponding district
def find_district(city_town):
    if city_town:
        city_town = city_town.lower()  # Convert city/town to lowercase for matching
        for district, towns in data.items():
            if city_town in towns:
                return district
    return None

# Take a single user input
user_input = input("Enter your details or describe something about a place: ")

# Limit the input to the first 70 characters for detection (you can change this value)
limited_input = user_input[:70]

# Detect city/town and predicted district using limited input
city_town = detect_city(limited_input)
predicted_district = find_district(city_town) if city_town else None

# Store the input and results in the DataFrame
new_entry = {
    "User Input": user_input,  # Store the full input
    "City/Town": city_town,
    "Predicted District": predicted_district
}
full_df = pd.concat([full_df, pd.DataFrame([new_entry])], ignore_index=True)

if predicted_district:
    print(f"User input categorized under District: {predicted_district}")
else:
    print("District not found.")

# Function to save the DataFrame to an Excel file
def save_to_excel(predicted_district):
    # Use the predicted district for the file name, if available
    if predicted_district:
        output_file = f"{predicted_district} District.xlsx"
    else:
        output_file = "UserInput_Output.xlsx"  # Default file name if no district is found
    full_df.to_excel(output_file, index=False)
    print(f"Data saved to {output_file}")

# Save the DataFrame to Excel after processing the input
save_to_excel(predicted_district)

Column names in the DataFrame: Index(['Districts', 'Cities/Towns/Mandals', 'count'], dtype='object')
Enter your details or describe something about a place: i am abhinaya sri swarna from piduguralla
User input categorized under District: Palnadu
Data saved to Palnadu District.xlsx
